- This notebook is a continuation of 'FM_ExperimentalPlan3_Concentration'
- Selected more samples for experiments to see if there are consistent trend between model prediction and experiments

In [1]:
import sys
sys.path.append("../src/models")
sys.path.append("../src")
sys.path.append("../")

In [2]:
import pandas as pd
import numpy as np
import pickle
import joblib
import matplotlib.pyplot as plt

from utils import *
from Concentration import *

In [3]:
all_results = joblib.load("../../data/Intermediate/results_generate_experimental_plan2_20250604.pkl")

In [4]:
all_results.keys()

dict_keys(['df_new_all0', 'df_new_all1', 'df_new_all2', 'results0', 'results1', 'results2', 'sample_special1', 'sample_special2', 'sample_special3', 'sample_smi_144', 'sample_smi_288', 'sample_dissmi_144', 'sample_dissmi_288'])

In [7]:
df_new_all0 = all_results['df_new_all0']
df_new_all1 = all_results['df_new_all1'] # replace similar
df_new_all2 = all_results['df_new_all2'] # replace dissimilar

In [ ]:
# Get samples needed

In [32]:
sample240_fmvsgp = extract_sample(df_new_all0, sample_idx=240, base_idx=240)

In [33]:
sample144_conc_sweep_ori = extract_sample(df_new_all0, sample_idx=144, base_idx=144)
sample144_conc_sweep_smi = extract_sample(df_new_all1, sample_idx=144, base_idx=144)
sample144_conc_sweep_dis = extract_sample(df_new_all2, sample_idx=144, base_idx=144)

In [34]:
sample288_conc_sweep_ori = extract_sample(df_new_all0, sample_idx=288, base_idx=288)
sample288_conc_sweep_smi = extract_sample(df_new_all1, sample_idx=288, base_idx=288)
sample288_conc_sweep_dis = extract_sample(df_new_all2, sample_idx=288, base_idx=288)

In [40]:
sample240_fmvsgp.keys()

dict_keys(['sample240_0', 'sample240_1', 'sample240_2', 'sample240_3', 'sample240_4', 'sample240_5', 'sample240_6'])

In [47]:
info_path = '../../data/Info_new_molecules.csv'
df_info = pd.read_csv(info_path)

In [48]:
datasets = (
    list(sample240_fmvsgp.values()) +
    list(sample144_conc_sweep_ori.values()) +
    list(sample144_conc_sweep_smi.values())[1:] +
    list(sample144_conc_sweep_dis.values())[1:] +
    list(sample288_conc_sweep_ori.values()) +
    list(sample288_conc_sweep_smi.values())[1:] +
    list(sample288_conc_sweep_dis.values())[1:] 
)

conc_cols = [f"conc{i}" for i in range(1, 8)]

# Convert conc columns in each dataset DataFrame
datasets_numeric = []
for df in datasets:
    df_copy = df.copy()
    df_copy[conc_cols] = df_copy[conc_cols].apply(pd.to_numeric, errors='coerce')
    datasets_numeric.append(df_copy)

# Process all datasets
original_concentrations = [
    extract_original_concentrations(df, df_info) for df in datasets_numeric
]

In [53]:
names = (
    # sample240 group
    [f"sample240_fmvsgp_{i}" for i in range(len(sample240_fmvsgp))] +

    # sample144 groups
    [f"sample144_ori_{i}" for i in range(len(sample144_conc_sweep_ori))] +
    [f"sample144_smi_{i}" for i in range(1, len(sample144_conc_sweep_smi))] +
    [f"sample144_dis_{i}" for i in range(1, len(sample144_conc_sweep_dis))] +

    # sample288 groups
    [f"sample288_ori_{i}" for i in range(len(sample288_conc_sweep_ori))] +
    [f"sample288_smi_{i}" for i in range(1, len(sample288_conc_sweep_smi))] +
    [f"sample288_dis_{i}" for i in range(1, len(sample288_conc_sweep_dis))]
)

import os

save_dir = "../../results/experimental_plan_conc_sweep/concentrations"
os.makedirs(save_dir, exist_ok=True)

for name, df in zip(names, original_concentrations):
    filename = os.path.join(save_dir, f"{name}_original_concentrations.csv")
    df.to_csv(filename, index=False)
    print(f"Saved: {filename}")

Saved: ../../results/experimental_plan_conc_sweep/concentrations/sample240_fmvsgp_0_original_concentrations.csv
Saved: ../../results/experimental_plan_conc_sweep/concentrations/sample240_fmvsgp_1_original_concentrations.csv
Saved: ../../results/experimental_plan_conc_sweep/concentrations/sample240_fmvsgp_2_original_concentrations.csv
Saved: ../../results/experimental_plan_conc_sweep/concentrations/sample240_fmvsgp_3_original_concentrations.csv
Saved: ../../results/experimental_plan_conc_sweep/concentrations/sample240_fmvsgp_4_original_concentrations.csv
Saved: ../../results/experimental_plan_conc_sweep/concentrations/sample240_fmvsgp_5_original_concentrations.csv
Saved: ../../results/experimental_plan_conc_sweep/concentrations/sample240_fmvsgp_6_original_concentrations.csv
Saved: ../../results/experimental_plan_conc_sweep/concentrations/sample144_ori_0_original_concentrations.csv
Saved: ../../results/experimental_plan_conc_sweep/concentrations/sample144_ori_1_original_concentrations.cs

In [54]:
save_objects_joblib(
    "experimental_plan3_concentrations_20251101.joblib",
    original_concentrations=original_concentrations,
)

In [55]:
len(original_concentrations)

45

In [60]:
save_dir = "../../results/experimental_plan_conc_sweep/dispense_volume"
os.makedirs(save_dir, exist_ok=True)


for i, df in enumerate(original_concentrations):
    solutes = df.columns
    dispense_df = get_dispense_volume(df, solutes, allow_zero=True)
    filename = os.path.join(save_dir, f"{names[i]}_dispense_volumes.csv")
    dispense_df.to_csv(filename, index=False)
    print(f"Saved: {names[i]}_dispense_volumes.csv")

Saved: sample240_fmvsgp_0_dispense_volumes.csv
Saved: sample240_fmvsgp_1_dispense_volumes.csv
Saved: sample240_fmvsgp_2_dispense_volumes.csv
Saved: sample240_fmvsgp_3_dispense_volumes.csv
Saved: sample240_fmvsgp_4_dispense_volumes.csv
Saved: sample240_fmvsgp_5_dispense_volumes.csv
Saved: sample240_fmvsgp_6_dispense_volumes.csv
Saved: sample144_ori_0_dispense_volumes.csv
Saved: sample144_ori_1_dispense_volumes.csv
Saved: sample144_ori_2_dispense_volumes.csv
Saved: sample144_ori_3_dispense_volumes.csv
Saved: sample144_ori_4_dispense_volumes.csv
Saved: sample144_ori_5_dispense_volumes.csv
Saved: sample144_ori_6_dispense_volumes.csv
Saved: sample144_smi_1_dispense_volumes.csv
Saved: sample144_smi_2_dispense_volumes.csv
Saved: sample144_smi_3_dispense_volumes.csv
Saved: sample144_smi_4_dispense_volumes.csv
Saved: sample144_smi_5_dispense_volumes.csv
Saved: sample144_smi_6_dispense_volumes.csv
Saved: sample144_dis_1_dispense_volumes.csv
Saved: sample144_dis_2_dispense_volumes.csv
Saved: samp

In [66]:
### Integrate dispense volume for all samples

In [67]:
path2 = '../../results/experimental_plan_conc_sweep/dispense_volume'
os.listdir(path2)

['sample288_ori_5_dispense_volumes.csv',
 'sample144_ori_6_dispense_volumes.csv',
 'sample288_ori_0_dispense_volumes.csv',
 'sample144_ori_3_dispense_volumes.csv',
 'sample288_dis_2_dispense_volumes.csv',
 'sample240_fmvsgp_6_dispense_volumes.csv',
 'sample144_dis_1_dispense_volumes.csv',
 'sample240_fmvsgp_3_dispense_volumes.csv',
 'sample144_dis_4_dispense_volumes.csv',
 'sample144_smi_4_dispense_volumes.csv',
 'sample288_smi_2_dispense_volumes.csv',
 'sample144_smi_1_dispense_volumes.csv',
 'sample288_dis_3_dispense_volumes.csv',
 'sample144_dis_5_dispense_volumes.csv',
 'sample240_fmvsgp_2_dispense_volumes.csv',
 'sample288_dis_6_dispense_volumes.csv',
 'sample288_ori_4_dispense_volumes.csv',
 'sample144_ori_2_dispense_volumes.csv',
 'sample288_ori_1_dispense_volumes.csv',
 'sample144_smi_5_dispense_volumes.csv',
 'sample288_smi_6_dispense_volumes.csv',
 'sample288_smi_3_dispense_volumes.csv',
 'sample144_smi_3_dispense_volumes.csv',
 'sample144_smi_6_dispense_volumes.csv',
 'sampl

In [69]:
folder_path = path2
save_path = '../../results/experimental_plan_conc_sweep/integrate'
os.makedirs(save_path, exist_ok=True)

files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]

# Reference file to get preferred column order
ref_file = 'sample240_fmvsgp_0_dispense_volumes.csv'
ref_df = pd.read_csv(os.path.join(folder_path, ref_file), sep='\t' if ref_file.endswith('.tsv') else ',')

# Get all unique columns from all files
all_columns = set(ref_df.columns)
for file in files:
    df = pd.read_csv(os.path.join(folder_path, file), sep='\t' if file.endswith('.tsv') else ',')
    all_columns.update(df.columns)

# Preserve reference column order and add any new ones at the end
ordered_columns = list(ref_df.columns) + sorted(all_columns - set(ref_df.columns))

# Collect all aligned rows
combined_rows = []
for file in files:
    df = pd.read_csv(os.path.join(folder_path, file), sep='\t' if file.endswith('.tsv') else ',')
    row = df.iloc[0] if not df.empty else pd.Series(dtype=float)
    row_dict = row.to_dict()
    aligned_row = {col: row_dict.get(col, 0.0) for col in ordered_columns}
    aligned_row['source_file'] = file
    combined_rows.append(aligned_row)

# Create final DataFrame
final_df = pd.DataFrame(combined_rows)
output_file = f"combined_sample_dispense_volumes_conc_sweep.csv"
final_df.to_csv(os.path.join(save_path, output_file), index=False)
final_df

/Users/haoliu/anaconda3/envs/ActiveLearning/lib/python3.11/site-packages/rdkit/Chem/PandasPatcher.py:132: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  for col in df_subset.columns[df_subset.applymap(MolFormatter.is_mol).any()]


,Decanoic acid (50 mM),Decanoic acid (10 mM),Decanoic acid (2 mM),Decanoate (50 mM),Decanoate (10 mM),Decanoate (2 mM),Decylamine (50 mM),Decylamine (10 mM),Decylamine (2 mM),Decyltrimethyl ammonium bromid (50 mM),...,Decanol (3 mM),Glycerol monodecanoate (10 mM),Glycerol monodecanoate (2 mM),Decanal (10 mM),Decanal (2 mM),Decanal (50 mM),Perfluorooctanoic acid (10 mM),Perfluorooctanoic acid (2 mM),Perfluorooctanoic acid (50 mM),source_file
0,0.0,8.8,0.0,0.0,10.1,0.0,0.0,9.8,0.0,17.2,...,0.0,17.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,sample288_ori_5_dispense_volumes.csv
1,0.0,19.6,0.0,0.0,7.4,0.0,0.0,0.0,14.1,0.0,...,0.0,0.0,6.2,0.0,0.0,0.0,0.0,0.0,0.0,sample144_ori_6_dispense_volumes.csv
2,0.0,8.8,0.0,0.0,10.1,0.0,0.0,9.8,0.0,0.0,...,0.0,17.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,sample288_ori_0_dispense_volumes.csv
3,0.0,19.6,0.0,0.0,7.4,0.0,0.0,0.0,14.1,6.9,...,0.0,0.0,6.2,0.0,0.0,0.0,0.0,0.0,0.0,sample144_ori_3_dispense_volumes.csv
4,0.0,8.8,0.0,0.0,10.1,0.0,0.0,9.8,0.0,0.0,...,0.0,17.0,0.0,0.0,0.0,0.0,19.0,0.0,0.0,sample288_dis_2_dispense_volumes.csv
5,9.3,0.0,0.0,9.1,0.0,0.0,7.4,0.0,0.0,0.0,...,3.1,5.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,sample240_fmvsgp_6_dispense_volumes.csv
6,0.0,19.6,0.0,0.0,7.4,0.0,0.0,0.0,14.1,0.0,...,0.0,0.0,6.2,0.0,0.0,0.0,7.9,0.0,0.0,sample144_dis_1_dispense_volumes.csv
7,9.3,0.0,0.0,9.1,0.0,0.0,7.4,0.0,0.0,6.9,...,3.1,5.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,sample240_fmvsgp_3_dispense_volumes.csv
8,0.0,19.6,0.0,0.0,7.4,0.0,0.0,0.0,14.1,0.0,...,0.0,0.0,6.2,0.0,0.0,0.0,0.0,0.0,11.2,sample144_dis_4_dispense_volumes.csv
9,0.0,19.6,0.0,0.0,7.4,0.0,0.0,0.0,14.1,0.0,...,0.0,0.0,6.2,0.0,0.0,11.2,0.0,0.0,0.0,sample144_smi_4_dispense_volumes.csv


In [70]:
final_df.to_csv('final_df_dispense_conc_sweep.csv', index=False)

In [71]:
import pandas as pd

df = final_df.copy()

# Adjust column order for specific compounds
# Define the correct order for the last compound-related columns
target_order = [
    'Decanal (50 mM)', 'Decanal (10 mM)', 'Decanal (2 mM)',
    'Perfluorooctanoic acid (50 mM)', 'Perfluorooctanoic acid (10 mM)', 'Perfluorooctanoic acid (2 mM)'
]

# Get existing columns except the reordered targets and 'source_file'
other_columns = [col for col in df.columns if col not in target_order and col != 'source_file']

# New column order
new_column_order = other_columns + target_order + ['source_file']

# Reorder columns
df = df[new_column_order]

names = (
    # sample240 group
    [f"sample240_fmvsgp_{i}" for i in range(len(sample240_fmvsgp))] +

    # sample144 groups
    [f"sample144_ori_{i}" for i in range(len(sample144_conc_sweep_ori))] +
    [f"sample144_smi_{i}" for i in range(1, len(sample144_conc_sweep_smi))] +  # skip first
    [f"sample144_dis_{i}" for i in range(1, len(sample144_conc_sweep_dis))] +  # skip first

    # sample288 groups
    [f"sample288_ori_{i}" for i in range(len(sample288_conc_sweep_ori))] +
    [f"sample288_smi_{i}" for i in range(1, len(sample288_conc_sweep_smi))] +  # skip first
    [f"sample288_dis_{i}" for i in range(1, len(sample288_conc_sweep_dis))]    # skip first
)

# Add suffix to match source_file naming pattern
desired_order = [f"{n}_dispense_volumes.csv" for n in names]

# Reorder rows based on desired source_file order
df['source_file'] = pd.Categorical(df['source_file'], categories=desired_order, ordered=True)
df = df.sort_values('source_file').reset_index(drop=True)

# Save updated DataFrame
output_file = f"combined_sample_dispense_volumes_conc_sweep.csv"
df.to_csv(os.path.join(save_path, output_file), index=False)

In [73]:
len(names)

45

In [74]:
df_name = pd.DataFrame(names, columns=["names"])
df_name.to_csv("../../data/Intermediate/names_exp3.csv", index=False)